# LLMs

Here we'll be interacting with a server that's exposing 2 LLMs via the runnable interface.

In [8]:
import os

# Steps to make the environment variable available in Notebook
# Add below line to your .bashrc or .zshrc then restart VScode
# export APIGEE_HOST='your-apigee-host-url'

# Get the APIGEE_HOST from environment variable
apigee_host = os.environ.get('APIGEE_HOST')

# Check if the environment variable is set
if apigee_host is None:
    raise ValueError("APIGEE_HOST environment variable is not set")

# Use apigee_host in your code
# print(f"Connecting to Apigee host: {apigee_host}")

In [9]:
from langchain.prompts.chat import ChatPromptTemplate

In [10]:
from langserve import RemoteRunnable

openai_llm = RemoteRunnable("http://localhost:8000/openai/")
openai_llm = RemoteRunnable("http://vm-pub-ip:8000/openai/")
openai_llm = RemoteRunnable(f"https://{apigee_host}/openai/")

Let's create a prompt composed of a system message and a human message.

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a highly educated person who loves to use big words. "
            + "You are also concise. Never answer in more than three sentences.",
        ),
        ("human", "how to use python to call openai api?"),
    ]
).format_messages()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a nice person who loves to keep the others happy "
            + "You are also creative and always answer in 300 words.",
        ),
        ("human", "tell a joke about the summer in 500 words?"),
    ]
).format_messages()

We can use either LLM

In [12]:
openai_llm.invoke(prompt)

AIMessage(content="Why did the sun go to school in the summer? To get a little brighter! \n\nSummer is a season full of warmth, relaxation, and fun in the sun. It's a time when the days are longer, the nights are shorter, and everyone seems to have a little extra pep in their step. From beach vacations to backyard barbecues, summer brings people together to enjoy the great outdoors and create lasting memories.\n\nOne of the best things about summer is the opportunity to cool off in the water. Whether it's swimming at the pool, splashing in the ocean, or floating down a lazy river, there's nothing quite like the feeling of refreshing water on a hot summer day. Some people even say that they're part fish during the summer because they spend so much time in the water!\n\nAnother highlight of summer is the delicious food that comes with it. From juicy watermelon to grilled burgers, summer is a time to indulge in all your favorite seasonal treats. There's something about eating outdoors tha

As with regular runnables, async invoke, batch and async batch variants are available by default

In [12]:
await openai_llm.ainvoke(prompt)

AIMessage(content="Why did the sun go to school in the summer?\n\nBecause it wanted to be a little brighter!\n\nSummer is a time of sunshine, fun, and laughter. It's a season where we get to enjoy the great outdoors, soak up the sun, and create wonderful memories with friends and family. The long days and warm nights of summer provide the perfect backdrop for all kinds of adventures and experiences.\n\nOne of the best things about summer is the opportunity to go on vacation and explore new places. Whether it's a beach trip, a camping adventure, or a visit to a new city, there are endless possibilities for summer travel. From building sandcastles on the beach to roasting marshmallows around a campfire, summer vacations are a time to relax, unwind, and make new memories.\n\nSummer is also a time for delicious food and refreshing drinks. From ice cream cones to barbecue cookouts, summer cuisine is all about enjoying fresh, seasonal ingredients and satisfying your taste buds. Whether you'r

Streaming is available by default

In [13]:
for chunk in openai_llm.stream(prompt):
    print(chunk.content, end="", flush=True)

Why did the sun go to school in the summer?

Because it wanted to be a little brighter!

Summer is such a wonderful time of year when the sun is shining, the birds are chirping, and the days are long and lazy. It's the perfect time to relax, have fun, and enjoy the great outdoors. Whether you're soaking up the sun at the beach, having a picnic in the park, or enjoying a BBQ with friends and family, there's nothing quite like the carefree vibes of summer.

One of the best things about summer is all the fun activities you can do. From swimming and surfing to hiking and camping, there's no shortage of ways to enjoy the warm weather. And let's not forget about all the delicious summer treats like ice cream, watermelon, and BBQ ribs. Yum!

But of course, with all the fun and sun comes the inevitable summer heat. Sometimes it can get so hot that you feel like you're melting! That's when it's time to break out the fans, crank up the AC, and maybe even take a dip in the pool to cool off. And l

In [14]:
async for chunk in openai_llm.astream(prompt):
    print(chunk.content, end="", flush=True)

I'm glad you're in the mood for some laughter! Here's a joke for you:

Why did the sun go to school in the summer?

Because it wanted to get a little brighter!

Summer is a wonderful time of year when the sun shines brightly, and everyone seems to be in a happier mood. It's a time for vacations, beach days, and enjoying the great outdoors. The warmth of the sun on your skin, the sound of waves crashing on the shore, and the smell of sunscreen in the air all contribute to the joyous feeling that summer brings.

One of the best things about summer is the opportunity to spend more time outside and soak up the sunshine. Whether you're lounging by the pool, having a picnic in the park, or going for a hike in the mountains, there's something truly special about being surrounded by nature in the summertime. The long days and warm nights make it the perfect season for making memories with friends and family.

Of course, with summer also comes the inevitable heatwaves and sunburns. It's all fun

In [18]:
from langchain.schema.runnable import RunnablePassthrough

In [19]:
comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
        ]
    )
    | openai_llm
)

joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | openai_llm
)

chain = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

In [20]:
chain.invoke({})

{'joke': AIMessage(content='Fact: The average person spends about 6 months of their lifetime waiting at red lights.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 42, 'total_tokens': 60}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6f7cafd2-b342-43fe-a3b3-2a21d6b1cbad-0', usage_metadata={'input_tokens': 42, 'output_tokens': 18, 'total_tokens': 60}),
 'classification': AIMessage(content='funny', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 190, 'total_tokens': 192}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b62d7217-5d8c-43b8-a7b8-199416d1bcf7-0', usage_metadata={'input_tokens': 190, 'output_tokens': 2, 'total_tokens': 192})}